# Домашняя работа №2

Используем PEP-8 проверку:

In [15]:
! pip install pycodestyle flake8 pycodestyle_magic
%load_ext pycodestyle_magic
%pycodestyle_on

Импортируем нужные модули:

In [16]:
! pip install pymystem3
! pip install pymorphy2
from pymystem3 import Mystem
m = Mystem()
from pprint import pprint
import json
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import collections

# №2

Замеряем время работы, анализируем текст, сохраняем в формате json:

In [ ]:
%%time
with open('silly_frenchman.txt', encoding='utf-8') as file:
    text = file.read()
    ana = m.analyze(text) #анализируем текст с помощью mystem
with open('mystem.json', 'w', encoding='utf-8') as file: 
    json.dump(ana, file, ensure_ascii = False) #сохраняем в json

# №3

Замеряем время работы, токенизируем, парсим текст, достаем лемму и часть речи из первого разбора для каждого слова, сохраняем в формате json:

In [ ]:
%%time
ana_mas = []
tok = word_tokenize(text) #токенизируем текст с помощью nltk
tok_filtered = [w.lower() for w in tok if w.isalpha()] #приводим слова к нижнему регистру
for tok in tok_filtered: 
    ana = morph.parse(tok) #парсим с помощью pymorphy
    ana_mas.append(ana) 
lem_pos_mas = []
for i in ana_mas:
    first = i[0] #берем первый разбор слова
    lem_pos = [first.normal_form, first.tag.POS] #берем из разбора лемму, часть речи
    lem_pos_mas.append(lem_pos)
with open('pymorphy.json', 'w', encoding='utf-8') as file: 
    json.dump(lem_pos_mas, file, ensure_ascii = False) #cохраняем в json

# №4

Преобразуем файл из предыдущего пункта в формат питона, создадим массив, где будут лежать части речи, счетчик с частями речи и их кол-вом, посчитаем число слов и долю каждой части речи от общего числа слов:

In [ ]:
lem_pos_mas = []
for line in open('pymorphy.json', encoding='utf-8'): #преобразуем в питоновский формат
    lem_pos_mas.extend(json.loads(line))
pos_mas = [] #тут части речи
for i in lem_pos_mas:
    pos = i[1] 
    pos_mas.append(pos) #добавляем части речи в массив
words_num = len(lem_pos_mas) #считаем число слов
pos_counter = Counter(pos_mas) #считаем число частей речи
pos_count_dict = dict(pos_counter) #преобразуем в словарь ("часть речи" - "кол-во")
for key in pos_count_dict.keys():
    #fraction = pos_count_dict[key]/words_num
    pos_count_dict[key] = pos_count_dict[key]/words_num #считаем, какую долю слов составляет каждая часть речи
print(pos_count_dict)

## Топ-20 глаголов и наречий

Добавляем глаголы в счетчик, выводим 20 самых популярных:

In [ ]:
verb_count = collections.Counter() #подключим счетчик глаголов
for i in lem_pos_mas:
    if i[1] == 'VERB':
        verb_count[i[0]] += 1 #если пометка глагола, добавляем в счетчик на первое место глагол
        
top_verb = verb_count.most_common(20)   #ищем 20 самых популярных глаголов
for i in top_verb:
    print(i[0], i[1])

Аналогично для наречий:

In [ ]:
adv_count = collections.Counter()
for i in lem_pos_mas:
    if i[1] == 'ADVB':
        adv_count[i[0]] += 1
        
top_adv = adv_count.most_common(20)      
for i in top_adv:
    print(i[0], i[1])

# №5

Оставляем только леммы из массива с леммами и частью речи и записываем их в отдельный массив, ищем би- и триграммы, считаем их кол-во, оставляем 25 самых популярных:

In [ ]:
lemms = []
for i in lem_pos_mas:
    lemms.append(i[0]) #вытаскиваем только леммы, добавляем их в массив
bi = nltk.bigrams(lemms) #генерируем биграммы
tri = nltk.trigrams(lemms) #генерируем триграммы
bi_count = collections.Counter(bi).most_common(25) #вводим счетчик биграмм, оставляем 25 самых частотных
tri_count = collections.Counter(tri).most_common(25) #вводим счетчик триграмм, оставляем 25 самых частотных

Выводим би- и триграммы:

In [ ]:
for i in bi_count:
    print(i[0])

In [ ]:
for i in tri_count:
    print(i[0])